# Lab 1: Tokenisation and embeddings

In this lab, you will build an understanding of how text can be transformed into representations that computers can process and learn from. Specifically, you will explore two key concepts: *tokenisation* and *embeddings*. Tokenisation splits text into smaller units such as words, subwords, or characters. Embeddings are dense, fixed-size vector representations of tokens in a continuous space.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## Part 1: Tokenisation

In the first part of the lab, you will code and analyse a tokeniser based on the Byte Pair Encoding (BPE) algorithm.

### Utility functions

The BPE tokeniser transforms text into a list of integers representing tokens. As a warm-up, you will implement two utility functions on such lists. To simplify things, we define a shorthand for the type of pairs of integers:

In [1]:
import  torch

In [2]:
Pair = tuple[int, int]

#### 🎈 Task 1.01: Counting pairs

Write a function that counts all occurrences of pairs of consecutive token IDs in a given list. The function should return a dictionary that maps each pair to its count. Skip counts that are zero.

In [3]:
#def count(ids: list[int]) -> dict[Pair, int]:
    # TODO: Replace the following line with your own code
 #   return {}

In [4]:
ids = [1, 2, 3, 2, 1, 2, 3, 1, 2, 3 ,1 , 1, 2, 2, 3, 1, 2]
#print(count(ids))

In [5]:
def count(ids: list[int]) -> dict[Pair, int]:
    pair_freqs = {}
    
    # Iterate through the list of token IDs
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i + 1])
        pair_freqs[pair] = pair_freqs.get(pair, 0) + 1
    
    return pair_freqs

# Example usage:
#ids = [1, 2, 3, 2, 1, 2, 3, 1]
print(count(ids))

{(1, 2): 5, (2, 3): 4, (3, 2): 1, (2, 1): 1, (3, 1): 3, (1, 1): 1, (2, 2): 1}


#### 🎈 Task 1.02: Replacing pairs

Write a function that replaces all occurrences of a specified pair of consecutive token IDs in a given list by a new ID. The function should return the modified list.

In [6]:
def replace(ids: list[int], pair: Pair, new_id: int) -> list[int]:
    # TODO: Replace the following line with your own code
    return []

In [7]:
def replace(ids: list[int], pair: Pair, new_id: int) -> list[int]:
    modified_ids = ids.copy()  # copy of the original list
    
    i = 0
    while i < len(modified_ids) - 1:
        if (modified_ids[i], modified_ids[i + 1]) == pair:
            # Replace pair with new ID
            modified_ids[i : i + 2] = [new_id]
        else:
            i += 1
    
    return modified_ids

pair = (1, 2)
new_id = 69
print(replace(ids, pair, new_id))

[69, 3, 2, 69, 3, 69, 3, 1, 69, 2, 3, 69]


### Encoding and decoding

The next cell contains the core code for the tokeniser in the form of a class `Tokenizer`. This class implements two methods: `encode()` converts an input text to a list of token IDs by exhaustively applying rules for merging pairs of consecutive IDs, and `decode()` reverses this process by looking up the tokens corresponding to the token IDs.

In [8]:
class Tokenizer:
    def __init__(self):
        self.merges = {}
        self.vocab = {i: bytes([i]) for i in range(2**8)}

    def encode(self, text):
        ids = list(text.encode("utf-8"))
        while True:
            counts = count(ids)
            mergeable_pairs = counts.keys() & self.merges.keys()
            if len(mergeable_pairs) == 0:
                break
            to_merge = min(mergeable_pairs, key=self.merges.get)
            ids = replace(ids, to_merge, self.merges[to_merge])
        return ids

    def decode(self, ids):
        return b"".join((self.vocab[i] for i in ids)).decode("utf-8")

#### 🎓 Task 1.03: Encoding and decoding

Explain how the code implements the BPE algorithm. Use the following steps to check your understanding:

**Step&nbsp;1.** Annotate the attributes and methods of the `Tokenizer` class with their Python types. In particular, what is the type of `self.merges`? Use the `Pair` shorthand.

**Step&nbsp;2.** Explain how the implementation chooses which merge rule to apply. Provide an example that illustrates the logic.

In [ ]:
#Anwers to the 1.03
#Step 1 and Step 2
It greedily chooses merge rules to apply based on which is added first.self.merge is a dict ,we have used pair in the code .


### Training a tokeniser

Upon initialisation, a tokeniser has an empty set of merge rules. Your next task is to complete the BPE algorithm and write code to learn these merge rules from a text.

#### 🎓 Task 1.04: Training a tokeniser

Write a function that induces a BPE tokeniser from a given text. The function should take the text (a string) and a target vocabulary size as input and return the trained tokeniser.

In [9]:
#def from_text(text: str, vocab_size: int) -> Tokenizer:
    # TODO: Replace the following line with your own code
#   return Tokenizer()

In [10]:
from collections import Counter
def from_text(text: str, vocab_size: int) -> Tokenizer:
    tokenizer = Tokenizer()
    ids = list(text.encode("utf-8"))
   
    counter = Counter(text)
    tokenizer.vocab = list(counter)
  #  ids = []
#for i in range(len(tokenizer.vocab):
  #      ids[i] = ord(tokenizer.vocab[i])
    new_idx = 256
    while len(tokenizer.vocab) < vocab_size:
        counts = count(ids)
        if not counts:
            break
        most_frequent = max(counts, key=counts.get)
        #print(most_frequent)
        tokenizer.merges[most_frequent] = new_idx
        tokenizer.vocab.append( most_frequent[0] + most_frequent[1])
        ids = replace(ids, most_frequent, new_idx)
        new_idx+=1
    return tokenizer

To help you test your implementation, we provide three text files together with tokenisers trained on these files. Each text file contains the first 1&nbsp;million Unicode characters in a language-specific Wikipedia:

| Text file | Tokeniser file | Wikipedia |
|---|---|---|
| `wiki-en-1m.txt` | `wiki-en-1m.tok` | [Simple English](https://simple.wikipedia.org/) |
| `wiki-is-1m.txt` | `wiki-is-1m.tok` | [Icelandic](https://is.wikipedia.org/) |
| `wiki-sv-1m.txt` | `wiki-sv-1m.tok` | [Swedish](https://sv.wikipedia.org/) |

A tokeniser file consists of lines specifying merge rules. For example, the first line in the tokeniser file for Swedish is `101 114`, which expresses that this rule combines the token with ID 101 (`e`) and the token with ID 114 (`r`). The ID of the new token (`er`) is 256 plus the (zero-indexed) line number on which the rule is found. The following code saves a `Tokenizer` to a file with this format:

In [11]:
#vargu125
def train_on_wikipedia(file_path: str, vocab_size: int) -> Tokenizer:
    #with open(file_path, "r", encoding="utf-8") as file:
        #text = file.read()
    text = open("wiki-en-1m.txt",'r').read()
    
    return from_text(text, vocab_size)



In [12]:
def save(tokenizer: Tokenizer, filename: str) -> None:
    with open(filename, "w") as f:
        for fst, snd in tokenizer.merges:
            print(f"{fst} {snd}", file=f)

save(train_on_wikipedia('wiki-en-1m.txt',256), "wikiOUT")

To test your code, compare the saved tokeniser to the provided tokeniser using `diff`.

### Tokenisation quirks

The tokeniser is a key component of language models, as it defines the minimal chunks of text the model can “see” and work with. As you will see in this section, tokenisation is also responsible for several deficiencies and unexpected behaviours of language models.

One helpful tool for experimenting with tokenisers in language models is the web app [Tiktokenizer](https://tiktokenizer.vercel.app/). This app lets you play around with, among others, [`cl100k_base`](https://tiktokenizer.vercel.app/?model=cl100k_base), the tokeniser used in the free version of ChatGPT and OpenAI’s APIs, and [`o200k_base`](https://tiktokenizer.vercel.app/?model=o200k_base), used in GPT-4o.

#### 🎓 Task 1.05: Tokenisation quirks

Prompt [ChatGPT](https://chatgpt.com/) to reverse the letters in the following words:

```
creativecommons
MERCHANTABILITY
NSNotification
authentication
```

How many of these words come out right? What could be the problem when words come out wrong? Generate ideas by inspecting the words in Tiktokenizer. Try to come up with other prompts that illustrate problems related to tokenisation.

![SNOWFALL](image.png) 

The Words show as one token in tiktokenizer. We had a theory that maybe chatgpt would have trouble with Composite Words, e.g. swimsuit, meatball, popcorn etc. However, it handled these well, even though they show up as >=2 tokens in tiktokenizer. However, chatgpt showed more issues when trying to reverse Words that were randomly capitalised, possibly because that increases the number of tokens further, as shown by tiktokenizer.

the image above tells the chatgpt reversal of words

### Tokenisation and multi-linguality

Many NLP systems and the tokenisers used with them are primarily trained on English data. In the next task, you will reflect on the effect this has when they are used to process non-English data.

The *context length* of a language model is the maximum number of preceding tokens the model can condition on when predicting the next token. This number is fixed and cannot be changed after training the model. For example, the context length of GPT-2 ([Radford et al., 2019](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)) is 1,024. 

While the context length of a language model is fixed, the amount of information that can be squeezed into this context length will depend on the tokeniser. Informally speaking, a model that needs more tokens to represent a given text cannot condition on as much information as one that needs fewer tokens.

#### 🎓 Task 1.06: Tokenisation and multi-linguality

Train a tokeniser on the English text file from Task&nbsp;1.04 and test it on the same text. How many tokens does it split the text into? Based on this, what is the expected number of Unicode characters of English text that can be fit into a context length of 1,024?

What do the numbers look like if you test the English tokeniser on the Icelandic text instead? What could explain the differences?

Interpreting the expected number of Unicode characters as a measure of representation efficiency, what do your results tell you about the efficiency of a language model primarily trained on English data when it is used to process non-English data? Why are these findings relevant?

## Part 2: Embeddings

In the second part of the lab, you will explore embeddings. An embedding layer is a network component that assigns each item in a finite set of elements (often called a *vocabulary*) a fixed-size vector. At first, these vectors are filled with random values, but during training, they are adjusted to suit the task at hand.

### Bag-of-words classifier

To help you build an intuition for embeddings and the vector representations learned by them, we will use a simple bag-of-words text classifier. The core part of this classifier only takes a few lines of code:

In [12]:
import torch.nn as nn


class Classifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.linear = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        return self.linear(self.embedding(x).mean(dim=-2))

#### 🎈 Task 1.07: Bag-of-words classifier

Explain how the bag-of-words classifier works. How does the code match the diagram you saw in the lectures? Why is there only one `nn.Embedding`, while the diagram shows three embedding layers? What does the keyword argument `dim=-2` do?

Solution for Task 1.07:
The bag-of-words classifier processes text by converting word indices into embeddings, averaging them, and passing the result through a linear layer for classification.

Code vs. Diagram: The lecture diagram shows three embedding layers, but the code uses only one nn.Embedding since PyTorch's embedding layer handles multiple words at once.
Role of dim=-2: The embedding layer outputs a tensor of shape (batch_size, num_words, embedding_dim). The .mean(dim=-2) operation averages embeddings across words, reducing the shape to (batch_size, embedding_dim), creating a single vector per sentence.

This simplifies the representation while maintaining efficiency, making it suitable for text classification tasks where word order is not important.

### Dataset

You will apply the classifier to a small dataset with Amazon customer reviews. This dataset is taken from [a much larger dataset](https://www.cs.jhu.edu/~mdredze/datasets/sentiment/) first described by [Blitzer et al. (2007)](https://aclanthology.org/P07-1056/).

The dataset contains whitespace-tokenised product reviews from two topics: cameras (`camera`) and music (`music`). Each review is additionally annotated for sentiment towards the product at hand: negative (`neg`) or positive (`pos`). The topic and sentiment labels are prepended to the review. As an example, here is the first review from the training data:

```
music neg oh man , this sucks really bad . good thing nu-metal is dead . thrash metal is real metal , this is for posers
```

The next cell contains a custom [`Dataset`](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class for the review dataset. To initialise an instance of this class, you specify the name of the file containing the reviews you want to load (`filename`) and which of the two labels you want to use (`label`): topic (0) or sentiment (1).

In [13]:
from torch.utils.data import Dataset


class ReviewDataset(Dataset):
    def __init__(self, filename: str, label: int = 0) -> None:
        with open(filename) as f:
            tokenized_lines = [line.split() for line in f]
        self.items = [(tokens[2:], tokens[label]) for tokens in tokenized_lines]

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, idx: int) -> tuple[list[str], str]:
        return self.items[idx]

### Vectoriser

To feed a review into the bag-of-words classifier, you first need to turn it into a vector of token IDs. Likewise, you need to convert the label (topic or sentiment) into an integer. The next cell contains a partially completed `ReviewVectoriser` class that handles this transformation.

In [14]:
#from collections import Counter

#import torch
#
# Type abbreviation for review–label pairs
#type Item = tuple[list[str], str]


#class ReviewVectorizer:
    PAD = "[PAD]"
    UNK = "[UNK]"

    #def __init__(self, dataset: ReviewDataset, n_vocab: int = 1024) -> None:
        # Unzip the dataset into reviews and labels
        reviews, labels = zip(*dataset)

        # Count the tokens and get the most common ones
        counter = Counter(t for r in reviews for t in r)
        most_common = [t for t, _ in counter.most_common(n_vocab - 2)]

        # Create the token-to-index and label-to-index mappings
        self.t2i = {t: i for i, t in enumerate([self.PAD, self.UNK] + most_common)}
        self.l2i = {l: i for i, l in enumerate(sorted(set(labels)))}

    #def __call__(self, items: list[Item]) -> tuple[torch.Tensor, torch.Tensor]:
        # TODO: Complete the implementation of this method
        return torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)

IndentationError: unexpected indent (526708284.py, line 10)

In [15]:
from collections import Counter

import torch

# Type abbreviation for review–label pairs
Item = tuple[list[str], str]


class ReviewVectorizer:
    PAD = "[PAD]"
    UNK = "[UNK]"

    def __init__(self, dataset: ReviewDataset, n_vocab: int = 1024) -> None:
        # Unzip the dataset into reviews and labels
        reviews, labels = zip(*dataset)

        # Count the tokens and get the most common ones
        counter = Counter(t for r in reviews for t in r)
        most_common = [t for t, _ in counter.most_common(n_vocab - 2)]

        # Create the token-to-index and label-to-index mappings
        self.t2i = {t: i for i, t in enumerate([self.PAD, self.UNK] + most_common)}
        self.l2i = {l: i for i, l in enumerate(sorted(set(labels)))}
    def __call__(self, items: list[Item]) -> tuple[torch.Tensor, torch.Tensor]:
        reviews, labels = zip(*items)
                
        # Convert reviews to lists of token IDs, replacing unknown tokens
        tokenized_reviews = [[self.t2i.get(t, self.t2i[self.UNK]) for t in review] for review in reviews]
        
        # Pad the tokenized reviews to the same length
        max_len = max(len(r) for r in tokenized_reviews)
        padded_reviews = [r + [self.t2i[self.PAD]] * (max_len - len(r)) for r in tokenized_reviews]
        
        # Convert labels to indices
        label_indices = [self.l2i[label] for label in labels]
        
        return torch.tensor(padded_reviews, dtype=torch.long), torch.tensor(label_indices, dtype=torch.long)


A `ReviewVectoriser` maps tokens and labels to IDs using two Python dictionaries. These dictionaries are set up when the vectoriser is initialised and queried when the vectoriser is called on a batch of review–label pairs. They include IDs for two special tokens:

`[PAD]` (Padding): Reviews can have different lengths, but PyTorch requires all vectors in a batch to be the same size. To handle this, the vectoriser adds `[PAD]` tokens to the end of shorter reviews so they match the length of the longest review in the batch.

`[UNK]` (Unknown): If a review contains a token that is not in the token-to-ID dictionary, the vectoriser assigns it the ID of the `[UNK]` token instead of a regular ID.

#### 🎓 Task 1.08: Vectoriser

Explain and complete the code of the vectoriser. Follow these steps:

**Step&nbsp;1.** Explain how unzipping works. What are the types of `reviews` and `labels`?

**Step&nbsp;2.** Explain how the token-to-ID and label-to-ID mappings are constructed. How does the `most_common()` method deal with elements that occur equally often?

**Step&nbsp;3.** Complete the implementation of the `__call__()` method. This method should convert a list of $m$ review–label pairs into a pair $(X, y)$ where $X$ is a matrix containing the vectors with token IDs for the reviews, and $y$ is a vector containing the IDs of the corresponding labels.

Solution for task 1.08 :::
In the ReviewVectorizer class, unzipping the dataset using zip(*dataset) separates the reviews and labels into two tuples: reviews, which contains lists of tokenized words, and labels, which holds the corresponding string labels. 

The token-to-ID mapping (t2i) is created by counting token frequencies using Counter, selecting the n_vocab - 2 most common tokens, and assigning each a unique index, along with special tokens [PAD] (for padding)&[UNK] (for unknown words). The label-to-ID mapping (l2i) assigns unique integer IDs to sorted unique labels. If multiple tokens have the same frequency, Python’s most_common() method returns them in order of frequency; in case of equal frequency, it returns the item that it outputs them in the order that they were encountered. 


The __call__() method processes a batch of reviews by converting tokens into integer sequences using t2i, replacing unknown words with [UNK], and padding shorter sequences to match the longest review. Labels are converted into numeric IDs using l2i. Finally, it returns two tensors: X, a padded matrix of tokenized reviews, and y, a vector of label IDs, enabling efficient batch processing for a machine learning model.

### Training the classifier

With the vectoriser completed, you are ready to train a classifier. More specifically, you can train two separate classifiers: one to predict the topic of a review, and one to predict the sentiment. The next cell contains a simple training loop that you can adapt for this purpose.

In [39]:
import torch.nn.functional as F


def train(filename: str="reviews-train.txt", learning_rate: float=0.001, batch_size: int=16, num_epochs: int=100):
    dataset = ReviewDataset(filename, label=0)
    processor = ReviewVectorizer(dataset, 1024)
    model = Classifier(1024, 64, len(processor.l2i))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=processor,
    )
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for bx, by in data_loader:
            optimizer.zero_grad()
            output = model(bx)
            loss = F.cross_entropy(output, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model

In [33]:
import torch.nn.functional as F


def train(filename: str="reviews-train.txt", learning_rate: float=0.001, batch_size: int=16, num_epochs: int=100):
    dataset = ReviewDataset(filename, label=1)
    processor = ReviewVectorizer(dataset, 1024)
    model = Classifier(1024, 64, len(processor.l2i))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=processor,
    )
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for bx, by in data_loader:
            optimizer.zero_grad()
            output = model(bx)
            loss = F.cross_entropy(output, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model

In [28]:
import torch.nn.functional as F


#def train():
    dataset = ReviewDataset("reviews-train.txt", label=0)
    processor = ReviewVectorizer(dataset, 1024)
    model = Classifier(1024, 64, len(processor.l2i))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=16,
        shuffle=True,
        collate_fn=processor,
    )
    #for epoch in range(10):
        model.train()
        running_loss = 0
        for bx, by in data_loader:
            optimizer.zero_grad()
            output = model(bx)
            loss = F.cross_entropy(output, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model

IndentationError: unexpected indent (1624771002.py, line 5)

In [29]:
#import torch.nn.functional as F


#def train():
   # dataset = ReviewDataset("reviews-train.txt", label=0)#loading the data file
   # processor = ReviewVectorizer(dataset, 1024)#passing the same dataset into review vectorizer with vocabsize as 1024
   # model = Classifier(1024, 64, len(processor.l2i))#this is a model that does the classification task with num_embedingsand embedding dimention,and no of classes 
   # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#initialise adam optimizer with learning rate 0.001 
   # data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=16,
        shuffle=True,
        collate_fn=processor,
   # )#passing the dataset into the dataloader function,with minibatch of 16, with our custom collate_fn from reviewvectorizer that pads etc, 
   # #and shuffling after all batches are done
   # for epoch in range(10):
   #     model.train()
   #    running_loss = 0
        for bx, by in data_loader:
    #        optimizer.zero_grad()
            output = model(bx)
     #       loss = F.cross_entropy(output, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    #    print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    #return processor, model

IndentationError: unexpected indent (1726152112.py, line 10)

#### 🎓 Task 1.09: Training loop

Explain the training loop. Follow these steps:

**Step&nbsp;1.** Go through the training loop line-by-line and add comments where you find it suitable. Your comments should be detailed enough for you to explain the main steps of the loop.

**Step&nbsp;2.** The training loop contains various hard-coded values like filename, learning rate, batch size, and epoch count. This makes the code less flexible. Revise the code so that you can specify these values using keyword arguments. Use the concrete values from the code as defaults.

#### 🎈 Task 1.10: Training the classifier

Adapt the next cell to train the classifier for the two prediction tasks. Based on the loss values, which task appears to be the harder one? What is the purpose of setting a seed?



In [40]:
torch.manual_seed(42)
vectorizer, model = train()

Epoch 0, loss: 0.6838
Epoch 1, loss: 0.6486
Epoch 2, loss: 0.6046
Epoch 3, loss: 0.5415
Epoch 4, loss: 0.4708
Epoch 5, loss: 0.4008
Epoch 6, loss: 0.3403
Epoch 7, loss: 0.2900
Epoch 8, loss: 0.2518
Epoch 9, loss: 0.2230
Epoch 10, loss: 0.1983
Epoch 11, loss: 0.1775
Epoch 12, loss: 0.1592
Epoch 13, loss: 0.1458
Epoch 14, loss: 0.1322
Epoch 15, loss: 0.1226
Epoch 16, loss: 0.1132
Epoch 17, loss: 0.1068
Epoch 18, loss: 0.0973
Epoch 19, loss: 0.0913
Epoch 20, loss: 0.0869
Epoch 21, loss: 0.0808
Epoch 22, loss: 0.0758
Epoch 23, loss: 0.0710
Epoch 24, loss: 0.0689
Epoch 25, loss: 0.0650
Epoch 26, loss: 0.0609
Epoch 27, loss: 0.0579
Epoch 28, loss: 0.0550
Epoch 29, loss: 0.0522
Epoch 30, loss: 0.0499
Epoch 31, loss: 0.0479
Epoch 32, loss: 0.0460
Epoch 33, loss: 0.0438
Epoch 34, loss: 0.0415
Epoch 35, loss: 0.0394
Epoch 36, loss: 0.0383
Epoch 37, loss: 0.0366
Epoch 38, loss: 0.0354
Epoch 39, loss: 0.0340
Epoch 40, loss: 0.0326
Epoch 41, loss: 0.0316
Epoch 42, loss: 0.0299
Epoch 43, loss: 0.029

### Inspecting the embeddings

Now that you have trained the classifier on two separate prediction tasks, it is interesting to inspect and compare the embedding vectors it learned in the process. For this you will use an online tool called the [Embedding Projector](http://projector.tensorflow.org). The next cell contains code to save the embeddings from a trained classifier in a format that can be loaded into this tool.

In [41]:
def save_embeddings(
    vectorizer: ReviewVectorizer,
    model: Classifier,
    vectors_filename: str,
    metadata_filename: str,
):
    i2t = {i: t for t, i in vectorizer.t2i.items()}
    embeddings = model.embedding.weight.detach().numpy()
    items = [(i2t[i], e) for i, e in enumerate(embeddings)]
    with open(vectors_filename, "wt") as f1, open(metadata_filename, "wt") as f2:
        for w, e in items:
            print("\t".join("{:.5f}".format(x) for x in e), file=f1)
            print(w, file=f2)

Call this code as follows:

In [42]:
save_embeddings(vectorizer, model, "vectors11(updated).tsv", "metadata11(updated)neww.tsv")

#### 🎓 Task 1.11: Inspecting the embeddings

Load the embeddings from the two classification tasks (topic classification and sentiment classification) into the Embedding Projector web app and inspect the vector spaces. How do they compare visually? Does the visualisation make sense to you?

The Embedding Projector offers visualisations based on three dimensionality reduction methods: [UMAP](https://umap-learn.readthedocs.io/en/latest/), [T-SNE](https://en.m.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding), and [PCA](https://en.m.wikipedia.org/wiki/Principal_component_analysis). Which of these seems most useful to you?

Focus on the embeddings for the words *repair* and *sturdy*. Are they close to each other or far away from another? What happens if you switch to the other task? How do you explain that?

Solution for 1.11::


Comparing repair and sturdy with only 10 epochs looked inconclusive. Their closest neighbours were also not clearly positive or negative. However, changing to 100 epochs for sentiment showed two clear clusters using UMAP, where sturdy was in the positive cluster (as evident by the positive Words in its neighbourhood) and vice versa. When looking at topics after 100epochs, sturdy and repair were in the same cluster, which makes sense as that was presumably the "camera" cluster. 



Comparing the dimensionality reduction methods, UMAP gave us the clearest clusters, so that was most useful for us.

### Initialisation of embedding layers

The error surfaces explored when training neural networks can be very complex. Because of this, it is crucial to choose “good” initial values for the parameters. In the final task of this lab, you will run a small experiment to see how alternative initialisations can affect a model’s performance.

In PyTorch, the weights of the embedding layer are initially set by sampling from the standard normal distribution, $\mathcal{N}(0, 1)$. However, research suggests other approaches may work better. For example, you have seen that embedding layers share similarities with linear layers, so it makes sense to use the same initialisation method for both. The default initialisation method for linear layers in PyTorch is the so-called Kaiming initialisation, introduced by [He et al. (2015)](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf).

#### 🎈 Task 1.12: Initialisation of embedding layers

Check the [source code of `nn.Linear`](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear) to see how PyTorch initialises the weights of linear layers using the Kaiming initialisation method. Apply the same method to the embedding layer of your classifier and see how this affects the loss of your model and the vector spaces.

In [54]:
import torch.nn as nn
import math


class Classifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        #initializing weights using kaiming distribution, as learnt from nn.linear
        self.embedding.weights = nn.init.kaiming_uniform(torch.empty(num_embeddings,embedding_dim), a=math.sqrt(5))
        self.linear = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        return self.linear(self.embedding(x).mean(dim=-2))

from torch.utils.data import Dataset


class ReviewDataset(Dataset):
    def __init__(self, filename: str, label: int = 0) -> None:
        with open(filename) as f:
            tokenized_lines = [line.split() for line in f]
        self.items = [(tokens[2:], tokens[label]) for tokens in tokenized_lines]

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, idx: int) -> tuple[list[str], str]:
        return self.items[idx]

from collections import Counter

import torch

# Type abbreviation for review–label pairs
Item = tuple[list[str], str]


class ReviewVectorizer:
    PAD = "[PAD]"
    UNK = "[UNK]"

    def __init__(self, dataset: ReviewDataset, n_vocab: int = 1024) -> None:
        # Unzip the dataset into reviews and labels
        reviews, labels = zip(*dataset)

        # Count the tokens and get the most common ones
        counter = Counter(t for r in reviews for t in r)
        most_common = [t for t, _ in counter.most_common(n_vocab - 2)]

        # Create the token-to-index and label-to-index mappings
        self.t2i = {t: i for i, t in enumerate([self.PAD, self.UNK] + most_common)}
        self.l2i = {l: i for i, l in enumerate(sorted(set(labels)))}
    def __call__(self, items: list[Item]) -> tuple[torch.Tensor, torch.Tensor]:
        reviews, labels = zip(*items)
                
        # Convert reviews to lists of token IDs, replacing unknown tokens
        tokenized_reviews = [[self.t2i.get(t, self.t2i[self.UNK]) for t in review] for review in reviews]
        
        # Pad the tokenized reviews to the same length
        max_len = max(len(r) for r in tokenized_reviews)
        padded_reviews = [r + [self.t2i[self.PAD]] * (max_len - len(r)) for r in tokenized_reviews]
        
        # Convert labels to indices
        label_indices = [self.l2i[label] for label in labels]
        
        return torch.tensor(padded_reviews, dtype=torch.long), torch.tensor(label_indices, dtype=torch.long)



import torch.nn.functional as F


def train(filename: str="reviews-train.txt", learning_rate: float=0.001, batch_size: int=16, num_epochs: int=100):
    dataset = ReviewDataset(filename, label=0)
    processor = ReviewVectorizer(dataset, 1024)
    model = Classifier(1024, 64, len(processor.l2i))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=processor,
    )
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for bx, by in data_loader:
            optimizer.zero_grad()
            output = model(bx)
            loss = F.cross_entropy(output, by)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model






#import torch.nn.functional as F


#def train(filename: str="reviews-train.txt", learning_rate: float=0.001, batch_size: int=16, num_epochs: int=100):
#    dataset = ReviewDataset(filename, label=1)
# #   processor = ReviewVectorizer(dataset, 1024)
#    model = Classifier(1024, 64, len(processor.l2i))
#    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#    data_loader = torch.utils.data.DataLoader(
#        dataset,
#        batch_size=batch_size,
#        shuffle=True,
#        collate_fn=processor,
#    )
#    for epoch in range(num_epochs):
#        model.train()
#        running_loss = 0
#        for bx, by in data_loader:
#            optimizer.zero_grad()
#            output = model(bx)
#            loss = F.cross_entropy(output, by)
#            loss.backward()
#            optimizer.step()
#            running_loss += loss.item()
#        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
#    return processor, model




torch.manual_seed(42)
vectorizer, model = train()



def save_embeddings(
    vectorizer: ReviewVectorizer,
    model: Classifier,
    vectors_filename: str,
    metadata_filename: str,
):
    i2t = {i: t for t, i in vectorizer.t2i.items()}
    embeddings = model.embedding.weight.detach().numpy()
    items = [(i2t[i], e) for i, e in enumerate(embeddings)]
    with open(vectors_filename, "wt") as f1, open(metadata_filename, "wt") as f2:
        for w, e in items:
            print("\t".join("{:.5f}".format(x) for x in e), file=f1)
            print(w, file=f2)






save_embeddings(vectorizer, model, "vectors1.12.tsv", "metadata1.12.tsv")

/tmp/ipykernel_303665/2181845924.py:9: FutureWarning: `nn.init.kaiming_uniform` is now deprecated in favor of `nn.init.kaiming_uniform_`.
  self.embedding.weights = nn.init.kaiming_uniform(torch.empty(num_embeddings,embedding_dim), a=math.sqrt(5))


Epoch 0, loss: 0.6826
Epoch 1, loss: 0.6470
Epoch 2, loss: 0.6042
Epoch 3, loss: 0.5436
Epoch 4, loss: 0.4727
Epoch 5, loss: 0.4053
Epoch 6, loss: 0.3472
Epoch 7, loss: 0.2968
Epoch 8, loss: 0.2600
Epoch 9, loss: 0.2267
Epoch 10, loss: 0.2034
Epoch 11, loss: 0.1819
Epoch 12, loss: 0.1640
Epoch 13, loss: 0.1511
Epoch 14, loss: 0.1371
Epoch 15, loss: 0.1257
Epoch 16, loss: 0.1163
Epoch 17, loss: 0.1077
Epoch 18, loss: 0.1026
Epoch 19, loss: 0.0934
Epoch 20, loss: 0.0893
Epoch 21, loss: 0.0838
Epoch 22, loss: 0.0785
Epoch 23, loss: 0.0753
Epoch 24, loss: 0.0697
Epoch 25, loss: 0.0666
Epoch 26, loss: 0.0629
Epoch 27, loss: 0.0600
Epoch 28, loss: 0.0573
Epoch 29, loss: 0.0545
Epoch 30, loss: 0.0512
Epoch 31, loss: 0.0498
Epoch 32, loss: 0.0467
Epoch 33, loss: 0.0455
Epoch 34, loss: 0.0434
Epoch 35, loss: 0.0425
Epoch 36, loss: 0.0393
Epoch 37, loss: 0.0377
Epoch 38, loss: 0.0366
Epoch 39, loss: 0.0356
Epoch 40, loss: 0.0341
Epoch 41, loss: 0.0324
Epoch 42, loss: 0.0305
Epoch 43, loss: 0.030

**🥳 Congratulations on finishing lab&nbsp;1!**

### Kaiming for the last epoch gave a loss of 0.0040, whereas the original version gave 0.0038. It seems that for our usecase the difference is quite minimal.